In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import os
import random

import tensorflow as tf
from keras import Input
from keras import optimizers
from keras.models import Sequential
from keras.layers import LSTM, Bidirectional, Dense, BatchNormalization, Conv1D, Conv2D, MaxPooling1D, Dropout, Flatten, Reshape
from keras.callbacks import EarlyStopping, ModelCheckpoint

import matplotlib.pyplot as plt

In [ ]:
def load_all_data_LSTM(inPath, SEQUENCE_N):
  # train_features, train_labels = np.array([[0]*feature_n]), np.array([[0]*label_n])
  train_features, train_labels = [], []

  file_paths = [os.path.join(inPath, f) for f in next(os.walk(inPath))[2] if f.endswith('.npy')]
  for fileIdx, path in enumerate(file_paths):    
    file_data = np.load(path, allow_pickle=True)

    # preprocess_data_LSTM
    seqImages = np.array([subsubdata[0].numpy()[0] for subdata in file_data for subsubdata in subdata], dtype=np.float32)
    seqLabels = np.array([subsubdata[1] for subdata in file_data for subsubdata in subdata], dtype=np.float32)
 
    # append splited sequence data
    sub_train_features, sub_train_labels = [], []
    # sub_train_features, sub_train_labels = np.array([[0]*feature_n]), np.array([[0]*label_n])
    for i in range(0, len(seqImages), SEQUENCE_N):
      sub_train_features.append(seqImages[i:i+SEQUENCE_N])
      sub_train_labels.append(seqLabels[i+SEQUENCE_N-1])
    train_features.extend( np.array(sub_train_features, dtype=np.float32) )
    train_labels.extend( np.array(sub_train_labels, dtype=np.float32) )

    #   sub_train_features = np.append(sub_train_features, seqImages[i:i+SEQUENCE_N], axis=0)
    #   sub_train_labels = np.append(sub_train_labels, seqLabels[i:i+SEQUENCE_N], axis=0)
    # train_features = np.append(train_features, sub_train_features, axis=0)
    # train_labels = np.append(train_labels, sub_train_labels, axis=0)

  # train_features, train_labels = np.array(train_features, dtype=np.float32), np.array(train_labels, dtype=np.float32)
  train_features, train_labels = np.array(train_features), np.array(train_labels)
  print(f'features shape: {train_features.shape}\nlabels shape: {train_labels.shape}')
  # return np.expand_dims(train_features[1:], 0), np.expand_dims(train_labels[1:], 0)
  return train_features, train_labels



# Generate class weights for imbalanced data
from collections import Counter
def getWeights(train_labels):
  labelLen = train_labels.shape[0]

  # onehotLabels = [np.where(onehotLabel==1)[0][0] for onehotLabel in train_labels]
  # labelWeightDict = {k: 1-(v/labelLen) for k,v in dict(Counter(onehotLabels)).items()}
  labelWeightDict  = {0: 0.5300869565217392, 1: 0.957391304347826, 2: 0.9426086956521739, 
                    3: 0.9464347826086956, 4: 0.9579130434782609, 5: 0.965391304347826, 
                    6: 0.9949565217391304, 7: 0.9928695652173913, 8: 0.7123478260869565}

  labelWeights = np.asarray([labelWeightDict[np.where(onehotLabel==1)[0][0]] for onehotLabel in train_labels])
  return labelWeights

input_n, feature_n, label_n = 500, 1536, 9
def get_LSTM(SEQUENCE_N):
  # LSTM input hidden layer
  model = Sequential()
  model.add(LSTM(feature_n, 
                #  input_shape = (timesteps, n_features),
                input_shape = (SEQUENCE_N, feature_n), 
                activation='relu',
                return_sequences=True))
  
  # normalization layer
  model.add(BatchNormalization())

  model.add(Dropout(0.3))
  
  # LSTM hidden layer
  model.add(LSTM(units=feature_n//2, activation='relu', return_sequences = True))
  model.add(Dropout(0.35))
  
  # LSTM hidden layer
  model.add(LSTM(units=feature_n//5, activation='relu', return_sequences = True))
  model.add(Dropout(0.4))
  
  # LSTM hidden layer
  model.add(LSTM(units=feature_n//10, activation='relu', return_sequences = False))
  # model.add(Dropout(0.45))
  
  # model.add(MaxPooling1D(pool_size=3, padding='same', data_format='channels_last'))
  # model.add(Conv1D(filters = feature_n//4,  # dimensionality of the output space
  #                  kernel_size = 3, # length of the convolution window
  #                  data_format = 'channels_last',
  #                  padding = 'valid', 
  #                  activation = 'relu'))

  #  Hidden layer
  # model.add(Flatten())
  # model.add(Dense(SEQUENCE_N*label_n, activation='relu'))

  # Softmax output layer 
  # model.add(Dense(feature_n, activation='softmax'))
  model.add(Dense(label_n, activation='softmax'))


  # model.add(Reshape((1, SEQUENCE_N, label_n)))

  # Cross Entropy objective function
  model.compile(loss = 'categorical_crossentropy',
                optimizer = tf.keras.optimizers.Adam(clipvalue=5, learning_rate=0.0001),
                # optimizer = 'adam', # ADAM optimization algorithm for speed
                # run_eagerly=True,
                metrics = ['accuracy']) 

  print(f'input shape: {model.input_shape}\noutput shape:{model.output_shape}')
  print(model.summary())

  return model
  
# def train(model, train_X, train_Y_onehot, epoch, batch=None):
def train(modelOutPath, tensorboard, model, train_X, train_Y_onehot, epoch, batch=1, valid_X=None, valid_Y=None):
  checkpointer = ModelCheckpoint(
                                # filepath=  os.path.join(outPath, f"model.{epoch:02d}-{val_loss:.2f}.hdf5"), 
                                filepath=modelOutPath,
                                verbose = 1, 
                                period=1, # number of epochs between checkpoints
                                # save_weights_only=True,
                                save_best_only = True,
                                monitor = 'loss', mode = 'min')

  earlystop = EarlyStopping(monitor='loss',
                            patience=10, 
                            verbose=1, 
                            mode="auto",
                            restore_best_weights=True)

  train_history = model.fit(train_X, train_Y_onehot,
                            batch_size = batch, 
                            epochs = epoch, 
                            shuffle = True,
                            sample_weight = getWeights(train_labels),
                            # validation_split = 0.2,
                            validation_data = (valid_X, valid_Y), 
                            callbacks = [checkpointer, earlystop, tensorboard],
                            )
  model.save(modelOutPath)
  print(f'Save model at {modelOutPath}')
  return train_history

def get_tensorboard(model, log_dir='CSCI527Project/Models/LSTM/tb_logs'):
  tensorboard = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir,
      histogram_freq=0,
      # batch_size=batch,
      # update_freq='batch',
      write_graph=True,
      write_grads=True
  )
  tensorboard.set_model(model)

  return tensorboard

def drawResult(history, epoch):
  result = {index:[] for index in history.keys()}
  for index, val in result.items():
    plt.plot(range(epoch), val, label=index)

  plt.xlabel('epoch'); plt.ylabel('error')
  plt.legend()

In [ ]:
inPath = 'CSCI527Project/Object detection features/training data'
inPath_valid =  'CSCI527Project/Object detection features/validation data'

epoch = 20

for SEQUENCE_N in [50]:
# for SEQUENCE_N in [50, 10]:
  modelOutPath = f'CSCI527Project/Models/LSTM/test_model_LSTM_epochs_{epoch}_seq_{SEQUENCE_N}.h5'
  log_dir = f'CSCI527Project/Models/LSTM/tb_logs_epochs_{epoch}_seq_{SEQUENCE_N}'

  # load LSTM data in sequences
  train_features, train_labels = load_all_data_LSTM(inPath, SEQUENCE_N)
  valid_features, valid_labels = load_all_data_LSTM(inPath_valid, SEQUENCE_N)
  print(train_features.shape, train_labels.shape)

  # train LSTM model
  model = get_LSTM(SEQUENCE_N)
  
  tensorboard = get_tensorboard(model, log_dir)
  %load_ext tensorboard
  %tensorboard --logdir 'CSCI527Project/Models/LSTM/tb_logs_epochs_10_seq_50'

  # history = train(model, train_features, train_labels, epoch)

  history = train(modelOutPath, tensorboard, 
                  model, train_features, train_labels, epoch, 
                  valid_X=valid_features, valid_Y=valid_labels, 
                  batch=16)
  # drawResult(history, epoch)

In [ ]:
epoch = 20
SEQUENCE_N = 50
modelOutPath = f'CSCI527Project/Models/LSTM/test_model_LSTM_epochs_{epoch}_seq_{SEQUENCE_N}.h5'
log_dir = f'CSCI527Project/Models/LSTM/tb_logs_epochs_{epoch}_seq_{SEQUENCE_N}'

model = get_LSTM(SEQUENCE_N)
tensorboard = get_tensorboard(model, log_dir)
%load_ext tensorboard
%tensorboard --logdir 'CSCI527Project/Models/LSTM/tb_logs_epochs_20_seq_50'


In [ ]:
# !pip install -U tensorboard
import tensorflow as tf
import datetime
from tensorboard.plugins.hparams import api as hp

# Launch tensorboard in url
def lunchTensorboard(log_dir):
  tensorboard_launch_cmd = f"tensorboard dev upload --logdir '{log_dir}'"; # added missing " at the end    
  !$tensorboard_launch_cmd

lunchTensorboard(log_dir)